In [1]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon

# Load the Shapefile
shapefile_path = '../Lower layer Super Output Areas (December 2001) Boundaries EW BFC/LSOA_Dec_2001_EW_BFC_2022_-3090597439272179543/LSOA_2001_EW_BFC_V2.shp'
lsoa_gdf = gpd.read_file(shapefile_path)

# Ensure the GeoDataFrame has the correct coordinate reference system (CRS)
lsoa_gdf = lsoa_gdf.to_crs(epsg=4326)


In [2]:
import pandas as pd

participants = pd.read_csv('../LSOA_participants_unique.csv', index_col=None)
LSOA_use = list(participants['LSOA_code'].unique())
participants.head()

,Unnamed: 0,LSOA_code,id,area,LSOA11NM,TCITY15CD,TCITY15NM,FID
0,1,E01000001,3930876,London,City of London 001A,J01000055,London,1.0
1,2,E01000001,4650325,London,City of London 001A,J01000055,London,1.0
2,3,E01000001,3118216,London,City of London 001A,J01000055,London,1.0
3,4,E01000001,2126547,London,City of London 001A,J01000055,London,1.0
4,5,E01000001,5323908,London,City of London 001A,J01000055,London,1.0


In [3]:
LSOA_london = list(participants[participants['TCITY15NM']=="London"]["LSOA_code"].unique())
len(LSOA_london)

2936

In [4]:
london = lsoa_gdf[lsoa_gdf['LSOA01CD'].isin(LSOA_london)]
london['area'] = london['geometry'].to_crs(epsg=3857).area
london = london[['LSOA01CD','LSOA01NM','geometry','area']]

/Users/yueli/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [5]:
london.head()

,LSOA01CD,LSOA01NM,geometry,area
0,E01000001,City of London 001A,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",3.349810e+05
1,E01000002,City of London 001B,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",5.891986e+05
2,E01000003,City of London 001C,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",1.523513e+05
3,E01000004,City of London 001D,"POLYGON ((-0.07891 51.52041, -0.07910 51.51971...",5.908310e+06
4,E01000005,City of London 001E,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",4.889038e+05


In [23]:
# Define the function to get historical OSM data from ohsome API
def get_walking_network(geometry, date="2009-01-01", filter_='type:way'):
    base_url = "https://api.ohsome.org/v1/elements/geometry"
    
    data = {
        "bpolys": geometry,
        "time": date,
        "filter": filter_
    }
    
    response = requests.post(base_url, data=data)
    
    if response.status_code == 200:
        features = response.json()['features']
        return features
    else:
        print(f"Error fetching data: {response.status_code}")
        print(response.json())
        return None
    
import geopandas as gpd
from shapely.geometry import shape, MultiPoint, MultiLineString, MultiPolygon, GeometryCollection

def count_intersections(geojson_features):
    # Convert the input list to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(geojson_features)
    
    intersection_count = 0
    
    for i in range(len(gdf)):
        geom_i = gdf['geometry'].iloc[i]
        
        # Validate and fix geometry i if necessary
        if not geom_i.is_valid:
            geom_i = geom_i.buffer(0)
        
        for j in range(i + 1, len(gdf)):
            geom_j = gdf['geometry'].iloc[j]
            
            # Validate and fix geometry j if necessary
            if not geom_j.is_valid:
                geom_j = geom_j.buffer(0)
            
            try:
                # Calculate the intersection between two geometries
                intersection = geom_i.intersection(geom_j)
                
                # Check the type of intersection and increment the counter if there's a valid intersection
                if isinstance(intersection, (MultiPoint, MultiLineString, MultiPolygon, GeometryCollection)):
                    intersection_count += 1
                elif hasattr(intersection, 'coords') and len(intersection.coords[:]) > 0:
                    intersection_count += 1
                    
            except Exception as e:
                pass
                # print(f"Error processing intersection between {i} and {j}: {e}")
    
    return intersection_count

def polygon_to_coord_list(polygon):
    if isinstance(polygon, Polygon):
        polygon = make_valid(polygon)
        coords = list(polygon.exterior.coords)
        coord_list = ",".join([f"{x},{y}" for x, y in coords])
        return [coord_list]
    elif isinstance(polygon, MultiPolygon):
        coord_list = []
        for poly in polygon.geoms:
            poly = make_valid(poly)
            coords = list(poly.exterior.coords)
            coord_list.append(",".join([f"{x},{y}" for x, y in coords]))
        return coord_list
    else:
        raise TypeError("Unsupported geometry type")

In [7]:
filters = [
"type:way and (highway=footway)"
]

In [8]:
# initiate an empty columns for result recording
import pandas as pd

var_names = ['walking_highway_intersections']

for var in var_names:
    london[var] = [pd.NA] * london.shape[0]


In [9]:
london.head()

,LSOA01CD,LSOA01NM,geometry,area,walking_highway_intersections
0,E01000001,City of London 001A,"POLYGON ((-0.09667 51.52027, -0.09666 51.52025...",3.349810e+05,<NA>
1,E01000002,City of London 001B,"POLYGON ((-0.08969 51.52069, -0.08973 51.52057...",5.891986e+05,<NA>
2,E01000003,City of London 001C,"POLYGON ((-0.09653 51.52295, -0.09647 51.52282...",1.523513e+05,<NA>
3,E01000004,City of London 001D,"POLYGON ((-0.07891 51.52041, -0.07910 51.51971...",5.908310e+06,<NA>
4,E01000005,City of London 001E,"POLYGON ((-0.07571 51.51575, -0.07542 51.51555...",4.889038e+05,<NA>


In [10]:
london.shape

(2936, 5)

In [11]:
for y in ['2019','2014','2012','2010']:
    london.to_csv('london_walking_intersections_'+y+'.csv')

In [24]:
import requests
from geopy.distance import geodesic
import re
from shapely.validation import make_valid
from tqdm import tqdm
from shapely import wkt

for y in ['2019','2014','2012','2010']:
    london = pd.read_csv('london_walking_intersections_'+y+'.csv')
    london['geometry'] = london['geometry'].apply(wkt.loads)
    for i, row in tqdm(london.iterrows(), total=london.shape[0]):
        geometry_list = polygon_to_coord_list(row['geometry'])
        for j in range(len(filters)):
            try:
                intersections = 0
                for geometry in geometry_list:
                    features = get_walking_network(geometry, y+'-01-01', filters[j])
                    intersections += count_intersections(features)
                london.at[i, var_names[j]] = intersections
            except Exception as e:
                print(f"Error fetching POI data for LSOA {row['LSOA01CD']}: {e}")
                london.at[i, var_names[j]] = -1
        # Save the updated DataFrame to the CSV file
        if i % 3 == 0:
            london.to_csv('london_walking_intersections_'+y+'.csv', index=False)
    london.to_csv('london_walking_intersections_'+y+'.csv', index=False)

100%|██████████| 2936/2936 [58:00<00:00,  1.19s/it] 


In [25]:
stats = []
for y in ['2010','2012','2014','2019']:
    mtc = pd.read_csv('london_walking_intersections_'+y+'.csv')
    year_stats = []
    for v in var_names:
        year_stats.append(mtc[v].sum())
    stats.append(year_stats)
stats = pd.DataFrame(stats)
stats.columns = var_names
    

In [26]:
stats.index = ['2010','2012','2014','2019']

In [27]:
stats

# results should delete:
# amenity_biergarten, shop_brewing_supplies, shop_cannabis

,walking_highway_intersections
2010,9300.0
2012,15112.0
2014,20999.0
2019,34547.0


In [18]:
# backup function

# import json
# from shapely.geometry import Polygon, MultiPolygon

# def polygon_to_geojson_feature_collection(polygon, region_id='Region'):
#     features = []

#     def format_coordinates(coords):
#         return [[list(coord) for coord in coords]]

#     if isinstance(polygon, Polygon):
#         coordinates = format_coordinates(polygon.exterior.coords)
#         feature = {
#             "type": "Feature",
#             "properties": {"id": f"{region_id} 1"},
#             "geometry": {
#                 "type": "Polygon",
#                 "coordinates": coordinates
#             }
#         }
#         features.append(feature)

#     elif isinstance(polygon, MultiPolygon):
#         for idx, poly in enumerate(polygon.geoms):
#             coordinates = format_coordinates(poly.exterior.coords)
#             feature = {
#                 "type": "Feature",
#                 "properties": {"id": f"{region_id} {idx + 1}"},
#                 "geometry": {
#                     "type": "Polygon",
#                     "coordinates": coordinates
#                 }
#             }
#             features.append(feature)
#     else:
#         raise TypeError("Unsupported geometry type")

#     feature_collection = {
#         "type": "FeatureCollection",
#         "features": features
#     }
#     return feature_collection


{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'id': 'Region 1'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.5704498471484643, 54.923952175382496], [-1.5704497077420054, 54.92395207963483], [-1.5701740091293421, 54.92406343507078], [-1.570067292381269, 54.92410507449073], [-1.5699574872283233, 54.92414791875118], [-1.5698547810752077, 54.92418799326264], [-1.569747889753728, 54.924229700081426], [-1.5697289359590474, 54.92423709530048], [-1.5696262192179127, 54.92428016742882], [-1.569531771414466, 54.92431977124733], [-1.5694002422589792, 54.924374924008276], [-1.5692680481964985, 54.92443035554523], [-1.5690005287472784, 54.92455891469998], [-1.568879389802446, 54.9246171303187], [-1.5688792645069296, 54.92461702832883], [-1.5688746180357085, 54.92461321871992], [-1.5688275856568914, 54.924574662304586], [-1.5682360406280935, 54.92408970825292], [-1.5679862770907975, 54.92390418146634], [-1.567864526165285, 54.92381374343603], [-1.567676365597

In [23]:
count_intersections(features)

  0%|          | 22/10779 [00:05<48:21,  3.71it/s]


KeyboardInterrupt: 

In [15]:
london

,Unnamed: 0,LSOA01CD,LSOA01NM,geometry,area,walking_highway_intersections
0,0,E01000001,City of London 001A,POLYGON ((-0.096673331375757 51.52027362452772...,3.349810e+05,106.0
1,1,E01000002,City of London 001B,POLYGON ((-0.0896947088045908 51.5206871841854...,5.891986e+05,173.0
2,2,E01000003,City of London 001C,POLYGON ((-0.0965307593953206 51.5229489233266...,1.523513e+05,11.0
3,3,E01000004,City of London 001D,POLYGON ((-0.0789101868202144 51.5204064571767...,5.908310e+06,1447.0
4,4,E01000005,City of London 001E,POLYGON ((-0.0757107886483168 51.5157497490281...,4.889038e+05,63.0
...,...,...,...,...,...,...
2931,23993,E01023994,Ashford 012A,POLYGON ((0.835814912743647 51.163744409642014...,3.175157e+07,NaN
2932,24029,E01024030,Ashford 008D,POLYGON ((0.8470910517428684 51.13284950871105...,2.492635e+06,NaN
2933,26850,E01026851,Norwich 008E,MULTIPOLYGON (((1.3038276491026355 52.62655980...,4.039804e+06,NaN
2934,30396,E01030397,Epsom and Ewell 006E,POLYGON ((-0.2260138030290731 51.3508820931176...,2.465671e+06,NaN
